<a href="https://colab.research.google.com/github/gupta-keshav/age_gender_classification/blob/master/boostingTreesAgeSubmit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/Adience/datasetAge.rar", outdir="/content")

     |████████████████████████████████| 81kB 2.3MB/s 
patool: Extracting /content/drive/My Drive/Adience/datasetAge.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/Adience/datasetAge.rar"
patool:     with cwd='/content'
patool: ... /content/drive/My Drive/Adience/datasetAge.rar extracted to `/content'.


'/content'

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, PReLU, ReLU
from keras import optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
np.random.seed(48)
from keras.applications import vgg19
from keras.models import Model
input_shape = (256, 256, 3)
vgg = vgg19.VGG19(include_top=False ,weights='imagenet', 
                                     input_shape=input_shape)   

Using TensorFlow backend.






80142336/80134624 [==============================] - 6s 0us/step








In [0]:
last = vgg.output

In [5]:
x = Flatten()(last)
x = Dropout(0.3)(x)
x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)
x = Dropout(0.65)(x)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
preds = Dense(8, activation = 'softmax')(x)

In [0]:
cnn_model = Model(vgg.input, preds)

In [8]:
import pandas as pd
layers = [(layer.output, layer.name, layer.trainable) for layer in cnn_model.layers]
pd.set_option('max_colwidth', -1)

pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,"Tensor(""input_1:0"", shape=(?, 256, 256, 3), dtype=float32)",input_1,False
1,"Tensor(""block1_conv1/Relu:0"", shape=(?, 256, 256, 64), dtype=float32)",block1_conv1,True
2,"Tensor(""block1_conv2/Relu:0"", shape=(?, 256, 256, 64), dtype=float32)",block1_conv2,True
3,"Tensor(""block1_pool/MaxPool:0"", shape=(?, 128, 128, 64), dtype=float32)",block1_pool,True
4,"Tensor(""block2_conv1/Relu:0"", shape=(?, 128, 128, 128), dtype=float32)",block2_conv1,True
5,"Tensor(""block2_conv2/Relu:0"", shape=(?, 128, 128, 128), dtype=float32)",block2_conv2,True
6,"Tensor(""block2_pool/MaxPool:0"", shape=(?, 64, 64, 128), dtype=float32)",block2_pool,True
7,"Tensor(""block3_conv1/Relu:0"", shape=(?, 64, 64, 256), dtype=float32)",block3_conv1,True
8,"Tensor(""block3_conv2/Relu:0"", shape=(?, 64, 64, 256), dtype=float32)",block3_conv2,True
9,"Tensor(""block3_conv3/Relu:0"", shape=(?, 64, 64, 256), dtype=float32)",block3_conv3,True


In [0]:
for i in range(17):
  cnn_model.layers[i].trainable = False

In [10]:
cnn_model.compile(optimizer = optimizers.Adam(lr=1e-4), loss= 'categorical_crossentropy',
              metrics = ['accuracy'])

In [11]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
        '/content/datasetAge/train',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')
validate_generator = train_datagen.flow_from_directory(
        '/content/datasetAge/test',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')



Found 13840 images belonging to 8 classes.
Found 3676 images belonging to 8 classes.


In [12]:
history = cnn_model.fit_generator(train_generator, steps_per_epoch=440, epochs=4, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/4
440/440 [==============================] - 208s 473ms/step - loss: 8852.7652 - acc: 0.3484 - val_loss: 172.6309 - val_acc: 0.4108
Epoch 2/4
440/440 [==============================] - 178s 404ms/step - loss: 172.0502 - acc: 0.4424 - val_loss: 171.2723 - val_acc: 0.4704
Epoch 3/4
440/440 [==============================] - 177s 401ms/step - loss: 171.0511 - acc: 0.4871 - val_loss: 170.5973 - val_acc: 0.4653
Epoch 4/4
440/440 [==============================] - 176s 401ms/step - loss: 169.8275 - acc: 0.5378 - val_loss: 169.8931 - val_acc: 0.4946


In [0]:
feature_model = Model(cnn_model.input, cnn_model.get_layer('flatten_1').output)

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

datagen = ImageDataGenerator()
batch_size = 32

def extract_features(directory, sample_count):
  features = np.zeros(shape=(sample_count, 32768))
  labels = np.zeros(shape=(sample_count))
  generator = datagen.flow_from_directory(directory, target_size = (256,256),
                                          batch_size = batch_size,
                                          class_mode = 'binary')
  i = 0
  for inputs_batch, labels_batch in generator:
    features_batch = feature_model.predict(inputs_batch)
    features[i*batch_size: (i+1)*batch_size] = features_batch
    labels[i*batch_size: (i+1)*batch_size] = labels_batch
    i = i + 1
    if i*batch_size >= sample_count:
      break
  return features, labels

In [21]:
train_dir = '/content/datasetAge/train'
train_size = 13840
train_features, train_labels = extract_features(train_dir, train_size)

Found 13840 images belonging to 8 classes.


In [22]:
test_dir = '/content/datasetAge/test'
test_size = 3676
test_features, test_labels = extract_features(test_dir, test_size)

Found 3676 images belonging to 8 classes.


In [0]:
import lightgbm as lgbm

In [0]:
model = lgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.7, importance_type='split', lambda_l1=0.0,
               lambda_l2=1.0, learning_rate=0.07, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=100, min_split_gain=0.0, n_estimators=500,
               n_jobs=-1, nthread=3, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [25]:
model.fit(train_features, train_labels)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.7, importance_type='split', lambda_l1=0.0,
               lambda_l2=1.0, learning_rate=0.07, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=100, min_split_gain=0.0, n_estimators=500,
               n_jobs=-1, nthread=3, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
predictions = model.predict(test_features)

In [27]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_labels, predictions))

0.5029923830250272
